## Objetivo

El dataset fue extrído de [Kaggle](https://www.kaggle.com/competitions/dog-breed-identification/data).

check: [link](https://towardsdatascience.com/image-classification-python-keras-tutorial-kaggle-challenge-45a6332a58b8)